# Conception BDD Normalisée et Conforme au RGPD (SIDORA AI)
---
## 

## CRUD pour verifier les donneés

In [19]:
# import
from datetime import datetime
import pandas as pd
from passlib.hash import argon2



from sqlalchemy import Column, Integer, String, Boolean, DateTime, ForeignKey, create_engine, Table
from sqlalchemy.orm import relationship, declarative_base, sessionmaker
from sqlalchemy.sql import func

from components.models import Client, DonnePersonnel, Commande, Produit, Promotion, Age, Region

In [15]:
# db_reader.py

engine = create_engine("sqlite:///BD_Ventes_de_jeux_video.bd") 

# session
Session = sessionmaker(bind=engine)
session = Session()

In [20]:
# Function pour creater un Client, Commande
def create_Client(age_id, region_id):
    try:
        session.add(Client(
        age_id = age_id,
        region_id = region_id,
        ))
        session.commit()
    except Exception as e:
        session.rollback()
        raise e
    
def create_donne_pers(login, mot_de_passe_hash):
    try:
        session.add(DonnePersonnel(
            login = login,
            mot_de_passe_hash = mot_de_passe_hash,
            date_suppression = None,
            anonymise = False,
            ))
        session.commit()
    except Exception as e:
        session.rollback()
        raise e

def create_commande(client_id, produit_id, nb_produit):
    try:
        session.add(Commande(
            client_id = client_id,
            produit_id = produit_id,
            nb_produit = nb_produit,
            ))
        session.commit()
    except Exception as e:
        session.rollback()
        raise e

In [58]:
# Function pour faire un roquet et pour retourner DataFrame
def read_table(table_class, limit=None, filter_exp=None):
    """
    table_class: class SQLAlchemy (par exemple.: Client, Commande)
    filter: opt
    limit: opt

    """
    query = session.query(table_class)
    
    if filter_exp is not None:
        query = query.filter(filter_exp)
    
    if limit:
        query = query.limit(limit)

    if query and (table_class is Client):
        session.query(Client).update(
            {Client.date_derniere_utilisation: func.now()},
            synchronize_session=False
        )
        session.commit()
    
    df = pd.read_sql(query.statement, session.bind)
    return df

def read_promo(limit=None, filter_exp=None):
    query = session.query(Promotion)
    
    if filter_exp:
        query = query.filter(filter_exp)
    if limit:
        query = query.limit(limit)

    for i in query:
        print(f"Promo {i.promotion_id} ({i.promotion_percent}%) apply to regions:")
        for reg in i.regions:
            print("----------", f"  - {reg.region_id} : {reg.region_nom}")
        print(f"Promo {i.promotion_id} ({i.promotion_percent}%) apply to produit: {i.produit_id})")

In [26]:
df_client = read_table(Client, limit=5)
df_client

,client_id,age_id,region_id,date_creation,date_derniere_utilisation
0,1,2,1,2025-12-01 13:31:33,2025-12-01 13:54:23
1,2,0,0,2025-12-01 13:31:33,2025-12-01 13:54:23
2,3,2,2,2025-12-01 13:31:33,2025-12-01 13:54:23
3,4,1,0,2025-12-01 13:31:33,2025-12-01 13:54:23
4,5,3,0,2025-12-01 13:31:33,2025-12-01 13:54:23


In [27]:
df_produit = read_table(Produit, limit=5)
df_produit

,produit_id,name,prix,year_n,platform_cod,genre_cod,publisher_cod
0,1,Wii Sports,35,0,0,0,0
1,2,Super Mario Bros.,146,1,1,1,0
2,3,Mario Kart Wii,31,2,0,2,0
3,4,Wii Sports Resort,27,3,0,0,0
4,5,Pokemon Red/Pokemon Blue,73,4,2,3,0


In [59]:
df_promo = read_promo(limit=5)
df_promo


Promo 1 (70%) apply to regions:
----------   - 1 : NA
Promo 1 (70%) apply to produit: 5511)
Promo 2 (50%) apply to regions:
----------   - 4 : Other
Promo 2 (50%) apply to produit: 6512)
Promo 3 (50%) apply to regions:
----------   - 3 : JP
Promo 3 (50%) apply to produit: 6564)
Promo 4 (10%) apply to regions:
----------   - 2 : EU
Promo 4 (10%) apply to produit: 8839)
Promo 5 (60%) apply to regions:
----------   - 2 : EU
Promo 5 (60%) apply to produit: 936)


In [ ]:
session.close()
engine.dispose()